# Model Time Series for COVID: Publish for app

This notebook use multistep time serie model  (predicting number of cases future next days).

It converts Tensorflow model into TensorFlow Lite to be able to use it in a Lambda fonction on AWS.

After that, the lite model is tested and publish on AWS

Finally, the lambda function is tested

## Convert model in TFlite

**IMPORTANT TO DO manually:**
- **Update TRAIN_SPLIT in my_helpers.model module**

### import

In [1]:
# data useful lib
import pandas as pd
import numpy as np

# helper lib
import shutil
import os, stat
import re
import datetime
import math

# read json from http
import json
import urllib.request

# read csv from http
import io
import requests

# model lib
import tensorflow as tf

# from project
from my_helpers.model import prepare_to_lambda, retrieve_from_lambda
from my_helpers.model import prepare_to_lambda_future
from my_helpers.model import create_list_past_hist, predict_list

### Definitions

In [2]:
PATH_TO_SAVE_DATA = "."
PATH_DF_POS_FR = PATH_TO_SAVE_DATA + '/' + 'df_pos_fr.csv'
PATH_DF_TEST_FR = PATH_TO_SAVE_DATA + '/' + 'df_test_fr.csv'
PATH_JSON_METEO_FR = PATH_TO_SAVE_DATA + '/' + 'data_meteo_fr.json'
PATH_DF_FEAT_FR = PATH_TO_SAVE_DATA + '/' + 'df_feat_fr.csv' 
PATH_GEO_DEP_FR = PATH_TO_SAVE_DATA + '/sources/geofrance/' + 'departments.csv'
PATH_MDL_SINGLE_STEP = PATH_TO_SAVE_DATA + '/' + "mdl_single_step_pos_fr"
PATH_MDL_MULTI_STEP = PATH_TO_SAVE_DATA + '/' + "mdl_multi_step_pos_fr"
PATH_MDL_MULTI_TFLITE = PATH_TO_SAVE_DATA + '/' + \
    'serverless/tensorflow_lite_on_aws_lambda'
PATH_MDL_MULTI_TFLITE_FILE = PATH_MDL_MULTI_TFLITE + '/' + \
    "converted_model.tflite"
PATH_SERVERLESS = PATH_MDL_MULTI_TFLITE + '/' + 'serverless.yml'

date_format = "%Y-%m-%d"

#NB_POS_DATE_MIN_DF_FEAT = 140734 # on 13/05/2020
NB_POS_DATE_MIN_DF_FEAT = 140227 # on 12/05/2020

URL_PREDICT = 'https://yl0910jrga.execute-api.us-east-2.amazonaws.com/dev/infer'

# model 
PAST_HISTORY = 14 # days used to predict next values in future
FUTURE_TARGET = 7 # nb predict days later
STEP = 1

# plot
NB_DAY_PLOT = FUTURE_TARGET*9

# train split
from my_helpers.model import TRAIN_SPLIT
print(f"TRAIN_SPLIT = {TRAIN_SPLIT}")

TRAIN_SPLIT = 223


### helper functions

In [3]:
# save file before update
def clean_file(path_file_name):
    '''
    Clean file already traited : rename file with date
    '''
    try:
        d = datetime.datetime.now()
        str_date = '_' + d.strftime("%Y%m%d_%H_%M_%S")
       
        res_re = re.search('\.\w+$', path_file_name)
        
        path_file_name_saved = \
            path_file_name[0:res_re.start()] + str_date + res_re.group(0)
         
        shutil.move(path_file_name, path_file_name_saved) 
        print('File {} moved!'.format(path_file_name_saved))
    except:
        print('File {} does not exist!'.format(path_file_name))

### load data

In [4]:
df_feat_fr = pd.read_csv(PATH_DF_FEAT_FR)
df_feat_fr.index = df_feat_fr["date"]
df_feat_fr["train"] = [True if I <= TRAIN_SPLIT else False \
                       for I in range(df_feat_fr.shape[0])]
df_feat_fr


,T_min,date,T_max,H_min,H_max,extrap,pos,age_pos,test,age_test,day_num,nb_cases,sum_cases,Rt,rate_pos,train
date,,,,,,,,,,,,,,,,
2020-05-13,284.926667,2020-05-13,290.505000,64.661017,88.135593,0.0,881,61.023837,39034,55.429856,3,141108,NaN,NaN,2.257007,True
2020-05-14,285.050000,2020-05-14,290.963333,59.406780,84.847458,0.0,979,60.426966,41971,54.826499,4,142087,NaN,NaN,2.332563,True
2020-05-15,285.308333,2020-05-15,291.920000,57.372881,82.966102,0.0,1021,60.042116,47665,54.380069,5,143108,NaN,NaN,2.142033,True
2020-05-16,284.956667,2020-05-16,293.500000,53.741379,86.534483,0.0,291,60.158076,16523,54.488471,6,143399,NaN,NaN,1.761181,True
2020-05-17,285.598333,2020-05-17,294.446667,49.879310,85.500000,0.0,139,61.568345,6603,58.292443,0,143538,NaN,NaN,2.105104,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31,279.726271,2020-12-31,285.392373,69.610169,92.050847,0.0,16015,48.586263,332662,47.453568,4,2589192,192170.0,1.148683,4.814196,False
2021-01-01,279.578333,2021-01-01,283.950000,71.100000,92.683333,0.0,1562,55.415493,32334,51.218346,5,2590754,176281.0,1.023533,4.830828,False
2021-01-02,279.173333,2021-01-02,283.438333,72.066667,89.916667,0.0,13727,46.572886,226475,48.801766,6,2604481,179385.0,1.017504,6.061155,False


### prepare features

In [5]:
features = df_feat_fr.copy().filter(items=['T_min', 'T_max', 'H_min',
                                           'H_max', 'pos', 'test', 'day_num',
                                          'age_pos', 'age_test'])
features

,T_min,T_max,H_min,H_max,pos,test,day_num,age_pos,age_test
date,,,,,,,,,
2020-05-13,284.926667,290.505000,64.661017,88.135593,881,39034,3,61.023837,55.429856
2020-05-14,285.050000,290.963333,59.406780,84.847458,979,41971,4,60.426966,54.826499
2020-05-15,285.308333,291.920000,57.372881,82.966102,1021,47665,5,60.042116,54.380069
2020-05-16,284.956667,293.500000,53.741379,86.534483,291,16523,6,60.158076,54.488471
2020-05-17,285.598333,294.446667,49.879310,85.500000,139,6603,0,61.568345,58.292443
...,...,...,...,...,...,...,...,...,...
2020-12-31,279.726271,285.392373,69.610169,92.050847,16015,332662,4,48.586263,47.453568
2021-01-01,279.578333,283.950000,71.100000,92.683333,1562,32334,5,55.415493,51.218346
2021-01-02,279.173333,283.438333,72.066667,89.916667,13727,226475,6,46.572886,48.801766


### Prepare dataset

In [6]:
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)
dataset = (dataset-data_mean)/data_std

In [7]:
dataset.shape[1]

9

In [8]:
PAST_HISTORY

14

In [9]:
dataset.shape[1]

9

### Load model

In [10]:
%%time
# reload best model
multi_step_model = tf.keras.models.load_model(PATH_MDL_MULTI_STEP)


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x6412a60d0> and <tensorflow.python.keras.layers.core.Dropout object at 0x6403b1d50>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x6412e5610> and <tensorflow.python.keras.layers.core.Dense object at 0x6412efb90>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x6412e5610> and <tensorflow.python.keras.layers.core.Dropout object at 0x6403b1d50>).
CPU times: user 2.09 s, sys: 135 ms, total: 2.22 s
Wall time: 2.38 s


In [11]:
multi_step_model.inputs[0].dtype

tf.float32

In [12]:
run_model = tf.function(lambda x: multi_step_model(x))
# This is important, let's fix the input size.
INPUT_SIZE = dataset.shape[1]
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([1, PAST_HISTORY, INPUT_SIZE],
                  multi_step_model.inputs[0].dtype))

# model directory.
MODEL_DIR = PATH_TO_SAVE_DATA + "/" + "keras_lstm"
multi_step_model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)

'''converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.allow_custom_ops=True'''

tflite_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./keras_lstm/assets


### Save model TFlite

In [13]:
clean_file(PATH_MDL_MULTI_TFLITE_FILE)

File ./serverless/tensorflow_lite_on_aws_lambda/converted_model_20210108_18_33_23.tflite moved!


In [14]:
open(PATH_MDL_MULTI_TFLITE_FILE, "wb").write(tflite_model)

6104

## Test converted model

### Predict with TF model (not-converted one)

#### Past days

In [15]:
# TENSORFLOW MODEL :
# prepare list of past histories
list_x = create_list_past_hist(dataset)
# predict
y_multi_pred = predict_list(list_x, multi_step_model)
# convert in positive cases
y_pos_pred = (y_multi_pred * data_std[4]) + data_mean[4] 
y_pos_pred

[160 - 174]
[167 - 181]
[174 - 188]
[181 - 195]
[188 - 202]
[195 - 209]
[202 - 216]
[209 - 223]
[216 - 230]
9


array([[48774.977 , 48387.04  , 40821.426 , 27992.297 , 12843.852 ,
        10868.523 , 34764.176 , 33760.992 , 33141.227 , 27341.578 ,
        18373.746 ,  7751.032 ,  6830.9785, 23531.566 , 19770.203 ,
        18980.756 , 15548.844 , 10960.39  ,  5768.2373,  5715.255 ,
        14484.893 , 12793.738 , 12350.097 , 10456.0205,  8089.126 ,
         5889.612 ,  6011.135 , 10196.099 , 12056.082 , 11511.448 ,
         9788.3545,  7339.3965,  5025.651 ,  5832.9087, 10988.544 ,
        13247.568 , 12426.34  , 10832.238 ,  8438.341 ,  5974.1606,
         6879.1694, 12358.41  , 16208.102 , 15215.77  , 12103.836 ,
         8854.375 ,  6047.71  ,  7148.1304, 13890.699 , 20261.703 ,
        19739.703 , 13066.605 ,  6835.974 ,   922.2119,  3882.7031,
        17998.76  , 21422.441 , 20446.512 , 19690.766 , 15778.641 ,
        11032.605 , 12902.533 , 23091.342 ]], dtype=float32)

#### Future days

In [16]:
# prepare data : very last days
x_for_future = np.array([dataset[-PAST_HISTORY:,:]]) 
# predict next days
y_future_pred = multi_step_model.predict(x_for_future)

### Predict with TFlite & Compare 

#### Past days

In [17]:
# CONVERTED LITE MODEL
# load 
interpreter = tf.lite.Interpreter(model_content=tflite_model)

# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    # predict with tensorflow model
    expected = multi_step_model.predict(x_multi)
    # predict with TFlite model
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Reload Tlite model

In [18]:
interpreter = tf.lite.Interpreter(model_path=PATH_MDL_MULTI_TFLITE_FILE)

In [19]:
PATH_MDL_MULTI_TFLITE_FILE

'./serverless/tensorflow_lite_on_aws_lambda/converted_model.tflite'

### Predict reloaded model

#### Past days

In [20]:
# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    
    expected = multi_step_model.predict(x_multi)
    
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


In [21]:
x_multi.shape

(1, 14, 9)

In [22]:
len(list_x)

9

## Update API lambda AWS

### API lambda simulate

#### Past days

In [23]:
dataset.shape

(237, 9)

In [24]:
json_list_list_x = prepare_to_lambda(dataset)
# simulate input to lambda (double dumps ? why ? i don't know yet)
json_list_list_x = json.dumps(json_list_list_x)
# simulate lambda

event = {"body": json_list_list_x}

[160 - 174]
[167 - 181]
[174 - 188]
[181 - 195]
[188 - 202]
[195 - 209]
[202 - 216]
[209 - 223]
[216 - 230]


In [25]:
# lambda code (file ./serverless/tensorflow-lite-on-aws-lambda/handler.py)
from serverless.tensorflow_lite_on_aws_lambda.handler import predict
context = None
response = predict(event, context)


INPUT : nb. arrays : 9 / arrays shape: (1, 14, 9)
OUTPUT : nb. arrays : 9 / arrays shape in list: (1, 7)


In [26]:
# Retrieve from lambda in App code
# input : response
y_multi_pred_out = retrieve_from_lambda(response)      
y_multi_pred_out.shape

(1, 63)

In [27]:
y_multi_pred

array([[ 2.6288004 ,  2.602255  ,  2.084565  ,  1.2067096 ,  0.17015088,
         0.03498564,  1.670087  ,  1.6014425 ,  1.5590339 ,  1.162183  ,
         0.54854345, -0.1783341 , -0.24129039,  0.90147626,  0.6440986 ,
         0.5900792 ,  0.3552447 ,  0.04127178, -0.3140103 , -0.31763572,
         0.28244194,  0.16672178,  0.13636486,  0.0067594 , -0.15519947,
        -0.305705  , -0.2973896 , -0.01102621,  0.11624643,  0.07897891,
        -0.03892681, -0.206501  , -0.36482304, -0.30958503,  0.04319825,
         0.1977759 ,  0.14158194,  0.03250274, -0.13130382, -0.29991964,
        -0.23799282,  0.13693373,  0.40035552,  0.3324535 ,  0.11951406,
        -0.10283599, -0.2948869 , -0.21958871,  0.24178323,  0.6777303 ,
         0.6420115 ,  0.18539323, -0.24094854, -0.645608  , -0.44303125,
         0.5228844 ,  0.75715584,  0.69037604,  0.6386628 ,  0.3709689 ,
         0.04621322,  0.1741663 ,  0.8713532 ]], dtype=float32)

In [28]:
y_multi_pred_out

array([[ 2.62880039,  2.60225487,  2.08456492,  1.2067095 ,  0.17015123,
         0.03498639,  1.67008746,  1.60144246,  1.55903399,  1.16218293,
         0.54854339, -0.17833431, -0.24129054,  0.90147632,  0.64409864,
         0.59007919,  0.35524473,  0.04127174, -0.3140105 , -0.31763598,
         0.28244191,  0.16672169,  0.13636482,  0.00675946, -0.15519939,
        -0.30570498, -0.2973896 , -0.01102633,  0.11624654,  0.07897896,
        -0.03892688, -0.20650113, -0.36482304, -0.30958506,  0.04319834,
         0.19777584,  0.14158192,  0.03250279, -0.13130376, -0.29991958,
        -0.23799281,  0.13693365,  0.40035552,  0.33245349,  0.11951405,
        -0.10283599, -0.29488692, -0.21958868,  0.24178326,  0.67773032,
         0.64201152,  0.18539321, -0.24094854, -0.64560807, -0.44303131,
         0.52288443,  0.75715578,  0.69037598,  0.63866282,  0.37096894,
         0.04621326,  0.17416631,  0.87135327]])

In [29]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### Future days

In [30]:
# Prepare data to lambda (future)
json_list_list_x = prepare_to_lambda_future(dataset)

# simulate lambda
json_list_list_x = json.dumps(json_list_list_x) # dumps again : I dont know why
event = {"body": json_list_list_x}
context = None
response = predict(event, context)
y_future_pred_out = retrieve_from_lambda(response)      
y_future_pred_out.shape

INPUT : nb. arrays : 1 / arrays shape: (1, 14, 9)
OUTPUT : nb. arrays : 1 / arrays shape in list: (1, 7)


(1, 7)

In [31]:
y_future_pred_out

array([[ 0.79977012,  0.68679988,  0.48965859,  0.1596892 , -0.21241954,
        -0.05220608,  0.767407  ]])

In [32]:
y_future_pred

array([[ 0.79977036,  0.68679994,  0.48965842,  0.15968896, -0.21241964,
        -0.05220603,  0.7674072 ]], dtype=float32)

In [33]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Update AWS Lambda with new model


This part does:
- Go to : ./serverless//tensorflow-lite-on-aws-lambda
    
- Execute : sls deploy -v

In [34]:
str_exe = '#!/bin/bash\n' + \
    'export PATH="/usr/local/bin:$PATH"\n' + \
    f'cd {PATH_MDL_MULTI_TFLITE}\n' + \
    'serverless deploy -v'
str_exe

'#!/bin/bash\nexport PATH="/usr/local/bin:$PATH"\ncd ./serverless/tensorflow_lite_on_aws_lambda\nserverless deploy -v'

In [35]:
open('deploy_serverless.sh', "w").write(str_exe)
os.chmod('deploy_serverless.sh', stat.S_IRWXU)

In [36]:
!cat ./deploy_serverless.sh

#!/bin/bash
export PATH="/usr/local/bin:$PATH"
cd ./serverless/tensorflow_lite_on_aws_lambda
serverless deploy -v

In [37]:
!./deploy_serverless.sh

Serverless: Generated requirements from /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda/requirements.txt in /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda/.serverless/requirements.txt...
Serverless: Using static cache of requirements found at /Users/gregory/Library/Caches/serverless-python-requirements/ef4e42eb03bbad46f74fee99a3c01f994e8119e7a557f947779093ab37b248d3_slspyc ...
Serverless: Packaging service...
Serverless: Excluding development dependencies...
Serverless: Injecting required Python packages to package...
Serverless: Uploading CloudFormation file to S3...
Serverless: Uploading artifacts...
Serverless: Uploading service tensorflow-lite-on-aws-lambda.zip file to S3 (18.4 MB)...
Serverless: Validating template...
Serverless: Updating Stack...
Serverless: Checking Stack update progress...
CloudF

### API AWS real Test

#### Past days

In [38]:
# prepare input
json_list_list_x = prepare_to_lambda(dataset)
# REQUEST
resp = requests.post(URL_PREDICT, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

[160 - 174]
[167 - 181]
[174 - 188]
[181 - 195]
[188 - 202]
[195 - 209]
[202 - 216]
[209 - 223]
[216 - 230]
status code :  200
[[[2.628800392150879, 2.602254867553711, 2.0845649242401123, 1.2067095041275024, 0.1701509952545166, 0.034986209124326706, 1.6700873374938965]], [[1.6014424562454224, 1.5590341091156006, 1.1621830463409424, 0.5485436320304871, -0.1783340722322464, -0.2412903606891632, 0.9014763236045837]], [[0.6440985798835754, 0.5900791883468628, 0.35524478554725647, 0.04127185791730881, -0.31401023268699646, -0.3176356554031372, 0.2824419438838959]], [[0.16672183573246002, 0.1363648921251297, 0.006759408861398697, -0.15519948303699493, -0.3057050108909607, -0.29738956689834595, -0.011026181280612946]], [[0.11624646186828613, 0.07897894084453583, -0.038926828652620316, -0.2065010815858841, -0.36482304334640503, -0.3095850646495819, 0.043198294937610626]], [[0.1977759301662445, 0.14158198237419128, 0.03250274807214737, -0.1313038021326065, -0.29991966485977173, -0.2379927933216

In [39]:
len(json_list_list_x)

23498

In [40]:
resp.json()

[[[2.628800392150879,
   2.602254867553711,
   2.0845649242401123,
   1.2067095041275024,
   0.1701509952545166,
   0.034986209124326706,
   1.6700873374938965]],
 [[1.6014424562454224,
   1.5590341091156006,
   1.1621830463409424,
   0.5485436320304871,
   -0.1783340722322464,
   -0.2412903606891632,
   0.9014763236045837]],
 [[0.6440985798835754,
   0.5900791883468628,
   0.35524478554725647,
   0.04127185791730881,
   -0.31401023268699646,
   -0.3176356554031372,
   0.2824419438838959]],
 [[0.16672183573246002,
   0.1363648921251297,
   0.006759408861398697,
   -0.15519948303699493,
   -0.3057050108909607,
   -0.29738956689834595,
   -0.011026181280612946]],
 [[0.11624646186828613,
   0.07897894084453583,
   -0.038926828652620316,
   -0.2065010815858841,
   -0.36482304334640503,
   -0.3095850646495819,
   0.043198294937610626]],
 [[0.1977759301662445,
   0.14158198237419128,
   0.03250274807214737,
   -0.1313038021326065,
   -0.29991966485977173,
   -0.2379927933216095,
   0.1369337

In [41]:
y_multi_pred_out = retrieve_from_lambda(resp)      
y_multi_pred_out.shape

(1, 63)

In [42]:
y_multi_pred_out

array([[ 2.62880039,  2.60225487,  2.08456492,  1.2067095 ,  0.170151  ,
         0.03498621,  1.67008734,  1.60144246,  1.55903411,  1.16218305,
         0.54854363, -0.17833407, -0.24129036,  0.90147632,  0.64409858,
         0.59007919,  0.35524479,  0.04127186, -0.31401023, -0.31763566,
         0.28244194,  0.16672184,  0.13636489,  0.00675941, -0.15519948,
        -0.30570501, -0.29738957, -0.01102618,  0.11624646,  0.07897894,
        -0.03892683, -0.20650108, -0.36482304, -0.30958506,  0.04319829,
         0.19777593,  0.14158198,  0.03250275, -0.1313038 , -0.29991966,
        -0.23799279,  0.13693374,  0.40035558,  0.33245349,  0.11951405,
        -0.10283601, -0.29488692, -0.21958862,  0.24178328,  0.67773032,
         0.64201158,  0.18539327, -0.24094845, -0.64560807, -0.44303125,
         0.52288449,  0.75715584,  0.69037604,  0.63866282,  0.37096891,
         0.04621322,  0.17416631,  0.87135333]])

In [43]:
y_multi_pred

array([[ 2.6288004 ,  2.602255  ,  2.084565  ,  1.2067096 ,  0.17015088,
         0.03498564,  1.670087  ,  1.6014425 ,  1.5590339 ,  1.162183  ,
         0.54854345, -0.1783341 , -0.24129039,  0.90147626,  0.6440986 ,
         0.5900792 ,  0.3552447 ,  0.04127178, -0.3140103 , -0.31763572,
         0.28244194,  0.16672178,  0.13636486,  0.0067594 , -0.15519947,
        -0.305705  , -0.2973896 , -0.01102621,  0.11624643,  0.07897891,
        -0.03892681, -0.206501  , -0.36482304, -0.30958503,  0.04319825,
         0.1977759 ,  0.14158194,  0.03250274, -0.13130382, -0.29991964,
        -0.23799282,  0.13693373,  0.40035552,  0.3324535 ,  0.11951406,
        -0.10283599, -0.2948869 , -0.21958871,  0.24178323,  0.6777303 ,
         0.6420115 ,  0.18539323, -0.24094854, -0.645608  , -0.44303125,
         0.5228844 ,  0.75715584,  0.69037604,  0.6386628 ,  0.3709689 ,
         0.04621322,  0.1741663 ,  0.8713532 ]], dtype=float32)

In [44]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### future days

In [45]:
# prepare input
json_list_list_x = prepare_to_lambda_future(dataset)
# REQUEST URL_PREDICT = 'https://yl0910jrga.execute-api.us-east-2.amazonaws.com/dev/infer' 
resp = requests.post(URL_PREDICT, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

status code :  200
[[[0.7997703552246094, 0.686799943447113, 0.48965853452682495, 0.15968912839889526, -0.21241939067840576, -0.05220573768019676, 0.7674073576927185]]]


In [46]:
y_future_pred_out = retrieve_from_lambda(resp)      
y_future_pred_out

array([[ 0.79977036,  0.68679994,  0.48965853,  0.15968913, -0.21241939,
        -0.05220574,  0.76740736]])

In [47]:
y_future_pred

array([[ 0.79977036,  0.68679994,  0.48965842,  0.15968896, -0.21241964,
        -0.05220603,  0.7674072 ]], dtype=float32)

In [48]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.
